In [1]:
#Allocate min number of 'facilities' such that each node or its neighbour shound have the 'facility'
#min J = sum over i (X(i))
#such that:
#          for all i: X(i) + sum_over j belongs_to L(i,j) (X(j)) >= 1

In [2]:
import matplotlib.pyplot as plt
#networkx for drawing graph plots
import networkx as nx
#Define graph
G = nx.random_regular_graph(3, 20, seed=None)
#pos for position of the nodes
pos = nx.spring_layout(G)
l1 = nx.draw(G, pos=pos, with_labels=True)
#labels = nx.draw_networkx_labels(G, pos=pos, font_color='white', font_weight='bold')
plt.show()

TypeError: '_AxesStack' object is not callable

<Figure size 432x288 with 0 Axes>

In [3]:
from pyomo.environ import *
import numpy as np
import random
import pandas as pd

In [13]:
model = AbstractModel()
#Number of nodes
model.N = Param(mutable=True, initialize=len(G.nodes))
#Set of nodes
model.i = RangeSet(0, model.N - 1)
model.j = Set(initialize=model.i)
#L is parameter showing if i is connected to j
#Connectivity Matrix
model.L = Param(model.i, model.j, default=0, mutable=True)
#X and y are decision variables
model.X = Var(model.i, domain=Binary)
model.y = Var(within=NonNegativeReals, initialize=0)

def rule_C1(model, i):
    return model.X[i] + sum(model.X[j] for j in model.j if value(model.L[i,j]) == 1) >= 1
model.C1 = Constraint(model.i, rule = rule_C1)

def rule_C2(model):
    return model.y >= sum(model.X[i] for i in model.i)
model.C2 = Constraint(rule=rule_C2)
model.OF = Objective(expr=model.y, sense=minimize)

In [14]:
#feed data into the model
Lines = {edg:1 for edg in G.edges}
data = {None: {
    'N': {None: len(G.nodes)},
    'L': Lines,}}
instance = model.create_instance(data)

    mutable Param).  The linkage between this RangeSet and the original source
    data will be broken, so updating the data value in the future will not be
    reflected in this RangeSet.  To suppress this warning, explicitly convert
    the source data to a constant type (e.g., float, int, or immutable Param)


In [15]:
#glpk for linear programming
opt = SolverFactory('glpk')
results = opt.solve(instance) #solves and updates instance

In [16]:
print(value(instance.y))
for i in instance.i:
    if value(instance.X[i]) > 0:
        print(i, value(instance.X[i]))

7.0
0 1.0
2 1.0
5 1.0
7 1.0
14 1.0
17 1.0
19 1.0


In [17]:
print("The solver returned a status of :" + str(results.solver.status))

from pyomo.opt import SolverStatus, TerminationCondition

if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print("this is feasible and optimal")
elif results.solver.termination_condition == TerminationCondition.infeasible:
    print("do something about it? or exit?")
else:
    #something else is wrong
    print(str(results.solver))

The solver returned a status of :ok
this is feasible and optimal


In [9]:
#instance.C1.print()

In [18]:
cvec = []
for i in G.nodes:
    cvec.append(value(instance.X[i]))
l1 = nx.draw(G, pos=pos, node_color=cvec, with_labels=True)
labels = nx.draw_networkx_labels(G, pos=pos, font_color='white', font_weight='bold')
plt.show()

TypeError: '_AxesStack' object is not callable

<Figure size 432x288 with 0 Axes>